# Mlflow call api Evaluation Instructions

This Jupyter notebook ([mlflow_call_api_test_example.ipynb](file:///Users/jlkj/study/LLM_Evaluation/mlflow/mlflow_call_api_test_example.ipynb)) is designed to evaluate an ECG (electrocardiogram) recognition API using MLflow's generative AI evaluation capabilities. Below is a detailed instruction on how to use this notebook:

## Overview

The notebook performs the following tasks:
1. Sets up MLflow tracking for experiment management
2. Defines utility functions for handling image data and JSON files
3. Creates a function to call the ECG recognition API
4. Implements a custom scorer to evaluate API results
5. Runs evaluation on a dataset of ECG images with expected results

## Prerequisites

- Python environment with required packages: `mlflow`, `requests`, `base64`, `json`
- Access to MLflow tracking server at `http://192.168.100.30:5000`
- Access to ECG recognition API at `http://192.168.100.24:9004/api/jl-hospital/ecg_recog`
- Dataset of ECG images in `./data/xindiantu/` directory
- Annotated dataset file at `annotated_data/ecg_dataset_annotated.json`

## Step-by-Step Instructions

### 1. Environment Setup
The first cell initializes MLflow tracking:
- Connects to the MLflow tracking server
- Sets up an experiment named "ecg_recog"
- Tags the run as "ecg api test"

### 2. Utility Functions
Two helper functions are defined:
- `image_to_base64(image_path)`: Converts JPG images to base64 encoded strings for API transmission
- `read_json_file(file_path)`: Reads and parses JSON data files

### 3. API Calling Function
The `call_ecg_api(image, **kwargs)` function:
- Takes an image filename as input
- Converts the image to base64 format
- Sends a POST request to the ECG recognition API
- Returns the JSON response from the API

### 4. Custom Scoring Function
The `ecg_result_match` scorer:
- Compares API outputs with expected results
- Calculates a score based on matching of 8 ECG parameters
- Returns a normalized score between 0 and 1

### 5. Dataset Loading
Loads the annotated ECG dataset from `ecg_dataset_annotated.json`.

### 6. Evaluation Execution
Runs the evaluation process:
- Uses `mlflow.genai.evaluate()` to test the API
- Iterates through all samples in the dataset
- Applies the custom scorer to compare results
- Logs metrics and results to MLflow

## Expected Output

After running the evaluation:
- You'll see printed comparisons between API outputs and expected values
- A final evaluation result showing the mean score (e.g., 0.99)
- Results will be viewable in the MLflow UI via the provided link

## Customization

To adapt this notebook for your use:
1. Update MLflow tracking URI if needed
2. Modify API endpoint URL
3. Adjust the scoring function based on your specific evaluation criteria
4. Update dataset paths as needed

The current implementation achieves ~99% accuracy on the ECG parameter matching task.

In [ ]:
import mlflow
from mlflow.genai import scorer

mlflow.set_tracking_uri("http://192.168.100.30:5000")
mlflow.set_experiment("ecg_recog")

# set each experiment run name
mlflow.set_tag("mlflow.runName", "ecg api test")

In [ ]:
## Image to Base64 function
import base64

def image_to_base64(image_path):
    """
    Convert a JPG image file to base64 encoded string
    
    Args:
        image_path (str): Path to the JPG image file
        
    Returns:
        str: Base64 encoded string of the image
    """
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
        return encoded_string.decode('utf-8')

In [ ]:
## Read JSON File
import json

def read_json_file(file_path):
    """
    Read and parse JSON data from a file
    
    Args:
        file_path (str): Path to the JSON file
        
    Returns:
        dict or list: Parsed JSON data
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format in '{file_path}': {e}")
        return None
    except Exception as e:
        print(f"Error reading file '{file_path}': {e}")
        return None

In [ ]:
import requests
def call_ecg_api(image: str, **kwargs) -> str:
    API_URL = "http://192.168.100.24:9004/api/jl-hospital/ecg_recog"  ## ecg_recog api endpoint
    HEADERS = {
        "Content-Type": "application/json"
    }
    image_path = f'./data/xindiantu/{image}'
    image_base64 = image_to_base64(image_path)
    input_json = {"imageUrl": image_base64}
    try:
        response = requests.post(
            API_URL,
            headers=HEADERS,
            json=input_json,
            timeout=100
        )
        response.raise_for_status()  # 检查HTTP错误
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API调用失败: {str(e)}")
        return None

In [ ]:
## difine scorer function
@scorer
def ecg_result_match(outputs: str, expectations: dict) -> bool:
    print(f"outputs: {outputs}, expectations: {expectations}")
    score = 0
    try:
        if outputs['HeartRate'] == expectations["expected_response"]["HeartRate"]:
            score += 1
        if outputs['PR_Interval'] == expectations["expected_response"]["PR_Interval"]:
            score += 1
        if outputs['QRS_Duration'] == expectations["expected_response"]["QRS_Duration"]:
            score += 1
        if outputs['QT'] == expectations["expected_response"]["QT"]:
            score += 1
        if outputs['QTc'] == expectations["expected_response"]["QTc"]:
            score += 1
        if outputs['AXIS'] == expectations["expected_response"]["AXIS"]:
            score += 1
        if outputs['RV5'] == expectations["expected_response"]["RV5"]:
            score += 1
        if outputs['SV1'] == expectations["expected_response"]["SV1"]:
            score += 1
        
        final_score = score / 8
    except:
        return (False, f"Error{e}")
    
    return final_score

In [ ]:
dataset = read_json_file("annotated_data/ecg_dataset_annotated.json")

In [ ]:
results = mlflow.genai.evaluate(
            data=dataset,
            predict_fn=call_ecg_api,
            scorers=[ecg_result_match
            ],
        )
print(results)